In [1]:
# import the standard modules from python
import numpy as np
import matplotlib.pyplot as plt
from typing import Type, Iterable

# import the neccesary pyquil modules
from pyquil.paulis import PauliSum, PauliTerm

from qaoa.cost_function import QAOACostFunctionOnWFSim
from pyquil.api import local_qvm, WavefunctionSimulator

# import the QAOAParameters that we want to demo
from qaoa.parameters import AlternatingOperatorsQAOAParameters, AbstractQAOAParameters

from vqe.optimizer import scipy_optimizer

In [2]:
# create a hamiltonian on 3 qubits with 2 coupling terms (and no bias term this time)
hamiltonian = PauliSum.from_compact_str("0.7*Z0*Z1 + 1.2*Z0*Z2")
print("hamiltonian =", hamiltonian)

hamiltonian = (0.7+0j)*Z0*Z1 + (1.2+0j)*Z0*Z2


In [3]:
timesteps = 2       # equivalent to p in Farhis original paper
myAlternatingParams = AlternatingOperatorsQAOAParameters\
            .linear_ramp_from_hamiltonian(hamiltonian, timesteps) 
print(myAlternatingParams)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: []
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [0.525 0.175]
	gammas_singles: [0.175 0.525]
	gammas_pairs: [0.175 0.525]



In [4]:
sim = WavefunctionSimulator()

In [5]:
cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                        params=myAlternatingParams,
                                        sim=sim,
                                        return_standard_deviation=True,
                                        noisy=False,
                                        log=[])
res = scipy_optimizer(cost_function, myAlternatingParams.raw(), epsilon=1e-3,
                      maxiter=500)
res


QVMError: The value of QVM::SIZE is 0, which is not The SIZE wasn't a multiple of 8..

The QVM returned the above error. This could be due to a bug in the server or a
bug in your code. If you suspect this to be a bug in pyQuil or Rigetti Forest,
then please describe the problem in a GitHub issue at:
    https://github.com/rigetti/pyquil/issues

The error message disappears if we add just a single bias term to the Hamiltonian, even if it has a coefficient of zero.

In [6]:
# create a hamiltonian on 3 qubits with 2 coupling terms and 1 bias term
hamiltonian = PauliSum.from_compact_str("0.7*Z0*Z1 + 1.2*Z0*Z2 + 0*Z0")
print("hamiltonian =", hamiltonian)

hamiltonian = (0.7+0j)*Z0*Z1 + (1.2+0j)*Z0*Z2 + 0j*Z0


In [7]:
myAlternatingParams = AlternatingOperatorsQAOAParameters\
            .linear_ramp_from_hamiltonian(hamiltonian, timesteps) 

cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                        params=myAlternatingParams,
                                        sim=sim,
                                        return_standard_deviation=True,
                                        noisy=False,
                                        log=[])
res = scipy_optimizer(cost_function, myAlternatingParams.raw(), epsilon=1e-3,
                      maxiter=500)
res

     fun: -1.7923939377641958
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 83
  status: 1
 success: True
       x: array([0.5361181 , 0.3067025 , 0.17611314, 0.41231927, 0.52557695,
       0.91919116])